<a href="https://colab.research.google.com/github/ConnorSapphire/opendi-genetic-prompt-algorithm/blob/main/Colaboratory_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/ga

/content/drive/MyDrive/ga


In [ ]:
# !pip3 install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip3 install -q -U git+https://github.com/huggingface/peft.git
# !pip3 install -q -U git+https://github.com/huggingface/accelerate.git
# !pip3 install cohere
# !pip3 install tiktoken
# !pip3 install openai

In [ ]:
!pip3 install -U sentence-transformers
!pip3 install fastapi
!pip3 install kaleido
!pip3 install python-multipart
!pip3 install -e git+https://github.com/hypercycle-development/pyhypercycle-aim.git#egg=pyhypercycle_aim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3f1ccc3704d53047805202259f16a62f51a375b6f6258f7a3d217bef95d0a372
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import time
from transformers import AutoTokenizer, T5Tokenizer, T5EncoderModel, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoModel, AutoModelForCausalLM
import torch
# import bitsandbytes
# import accelerate
from sentence_transformers import SentenceTransformer
from pprint import pprint
import requests
# from openai import OpenAI

In [ ]:

def clean_prompt(prompt: str):
    """
    Cleans the propt to before passing to the model. Removes extra spaces and '.'
    """
    new_prompt = prompt
    new_prompt = re.sub(r"([A-Z])\.", r"\1", new_prompt)
    new_prompt = re.sub(r"[\n]{1,}", "\n", new_prompt)
    return new_prompt

def extract_answer(s: str):
    answer_match = [i for i in re.finditer(r"answer is (yes|no)\W", s, re.IGNORECASE)]
    if len(answer_match) > 0:
        answer_match = answer_match[-1]
        answer_string = s[answer_match.start(): answer_match.end()]
        # print(answer_string)
        matches = list(re.finditer(r"Yes|No", answer_string, re.IGNORECASE))[-1]
        return answer_string[matches.start(): matches.end()]
    else:
        return None

def encode_answer(a):
    if a is None:
      return None
    a = a.lower()
    if a == "yes":
        return True
    else:
        return False

def get_model(checkpoint, encoder = False):
    if encoder:
        return SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    else:
        return AutoModelForSeq2SeqLM.from_pretrained(checkpoint, max_length = 1024 + 512)

def get_tokenizer(checkpoint):
    return AutoTokenizer.from_pretrained(checkpoint)

parameters = {
    "max_new_tokens": None,
    "return_full_text": False
}

In [ ]:
import numpy as np

class Template:
    def __init__(self, template = None, demo_template = None):
        if template:
            try:
                self.template = open(template).read()
                self.demo_template = open(demo_template).read()
            except FileNotFoundError:
                print("File not found. Using the provided strings as templates")
                self.template = template
                self.demo_template = demo_template
        else:
            self.template = None

    def populate_single_demo(self, demo):
        if self.demo_template:
            new_demo = self.demo_template.replace("[question]", demo["question"])
            new_demo = new_demo.replace("[answer]", demo["answer"])
            return new_demo
        return f"Question: {demo['question']}\nAnswer: {demo['answer']}"

    def populate_multiple_demos(self, demos):
        return "\n\n".join([self.populate_single_demo(demo) for demo in demos])

    def get_prompt(self, instruction, demos = [], question = "", context = ""):
        if len(demos) == 0:
            demo_string = ""
        else:
            demo_string = self.populate_multiple_demos(demos)
        ip = self.template
        # print(instruction, demo_string, question, context)
        ip = ip.replace("[instruction]", instruction).replace("[demos]", demo_string).replace("[question]", question).replace("[context]", context)
        return ip

    def get_llm_input(self, question = "", instruction = "", demos = [], context = ""):
        return self.get_prompt(instruction, demos, question, context = context)


In [ ]:
template = "./templates/prompt/mutator"
demo_template = "./templates/demo/mutator"
test_template = Template(template, demo_template)
instruction = "afkadnf"
demos = [
    {"question": "akfkjf", "answer": "akbfkjef"},
    {"question": "kjenvjsa", "answer": "anfasn"}
]
question = "abfkwnqoinorinq"
print(test_template.get_llm_input(demos = demos, question = question))


You are a professional sentence rephraser. Your job is to take a given sentence, and produce a new sentence that is easier for the language models to understand.

Here are some examples of how rephrasing the sentence can improve the performance of the model.

Original: akfkjf
Rephrased: akbfkjef

Original: kjenvjsa
Rephrased: anfasn

Rephrase the next sentence to enhance the performance of the language model.
Original: abfkwnqoinorinq
Rephrased: 


In [ ]:
# template = PromptTemplate(
    # )

In [ ]:
import re
import torch
import torch.nn.functional as F

class Mutator:
    def __init__(self, model = "t5-small", alpha = 0.1, lamda = 1, p = 0.5, threshold = 0.4, max_length = 1024, combine_prob = 0.3):
        self.history = []
        self.lamda = lamda
        self.prob_select = p
        self.alpha = alpha
        self.threshold = threshold
        self.combine_prob = combine_prob
        self.seed = 0
        self.max_length = max_length
        self.supported_models = ["llama-small", "falcon-7b", "t5-base", "t5-large", "chatgpt", "t5-small"]
        if model not in self.supported_models:
            raise ValueError("Model not supported. Supported models:", self.supported_models)
        if model == "t5-base":
            # self.model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", max_length = self.max_length)
            checkpoint = "google/flan-t5-base"
        elif model == "t5-large":
            # self.model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", max_length = self.max_length)
            checkpoint = "google/flan-t5-large"
        elif model == "t5-small":
            checkpoint = "google/flan-t5-small"
        elif model == "chatgpt":
            checkpoint = "humarin/chatgpt_paraphraser_on_T5_base"
        elif model == "falcon-7b":
            checkpoint = "tiiuae/falcon-7b"
        self.model = get_model(checkpoint)
        self.tokenizer = get_tokenizer(checkpoint)
        self.encoder = get_model("", encoder = True)

        self.instruction = "You are a professional sentence rephraser. Your job is to take a given sentence, and produce a new sentence that is easier for the language models to understand.\n\nHere are some examples of how rephrasing the sentence can improve the performance of the model."
        self.demos = [
            {
                "question": "So, it is true that Lesley is a great-grandfather of Leroy",
                "answer": "Therefore Leslie is Leroy’s great-grandfather"
            }
        ]

        self.ques_start = "Rephrase the next sentence to enhance the performance of the language model."

    def clean_prompt(self, prompt: str):
        new_prompt = prompt
        new_prompt = re.sub(r"([A-Z])\.", r"\1", new_prompt)
        return new_prompt

    def split_into_sentences(self, text):
        alphabets= "([A-Za-z])"
        prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
        suffixes = "(Inc|Ltd|Jr|Sr|Co)"
        starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
        acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
        websites = "[.](com|net|org|io|gov|edu|me)"
        digits = "([0-9])"
        multiple_dots = r'\.{2,}'

        text = " " + text + "  "
        text = text.replace("\n"," ")
        text = re.sub(prefixes,"\\1<prd>",text)
        text = re.sub(websites,"<prd>\\1",text)
        text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
        text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
        if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
        text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
        text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
        text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
        text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
        text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
        text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
        text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
        if "”" in text: text = text.replace(".”","”.")
        if "\"" in text: text = text.replace(".\"","\".")
        if "!" in text: text = text.replace("!\"","\"!")
        if "?" in text: text = text.replace("?\"","\"?")
        text = text.replace(".",".<stop>")
        text = text.replace("?","?<stop>")
        text = text.replace("!","!<stop>")
        text = text.replace("<prd>",".")
        sentences = text.split("<stop>")
        sentences = [s.strip() for s in sentences]
        if sentences and not sentences[-1]: sentences = sentences[:-1]
        return sentences

    def extract_sentences(self, prompt: str, remove_tags = False):
        new_prompt = clean_prompt(prompt)
        question_idx = [m.start() for m in re.finditer("Question:", new_prompt)]
        option_idx = [m.start() for m in re.finditer("Options:", new_prompt)]
        answer_idx = [m.start() for m in re.finditer("Answer:", new_prompt)]
        if remove_tags:
            new_prompt = new_prompt.replace("Question: ", "")
            new_prompt = new_prompt.replace("Answer: ", "")
        return self.split_into_sentences(new_prompt)

    def clean_sentences(self, sentences):
        sts = sentences.copy()
        sts = [s.strip() for s in sts]
        return sts

    def get_sentences(self, prompt, remove_tags = False):
        sentences = self.extract_sentences(prompt, remove_tags = remove_tags)
        return self.clean_sentences(sentences)

    def get_bandit_score(self, encoded_sentence, w, ainv):
        return np.dot(encoded_sentence, w) + self.alpha * np.sqrt(np.dot(encoded_sentence, np.dot(ainv, encoded_sentence)))

    def encode_sentences(self, sentences):
        return self.encoder.encode(sentences)

    def get_demo(self, h):
        if h["score"] > 0:
            return {
                "question": h["before"],
                "answer": h["after"]
            }
        else:
            return {
                "question": h["after"],
                "answer": h["after"]
            }

    def get_sentence_distance(self, a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    def get_relevant_demos(self, sentence):
        if len(self.history) < 1:
            relevant_history = []
        else:
            # print("reld1")
            encoded_sentence = self.encode_sentences(sentence)
            prev_sentences = [h["before"] for h in self.history]
            # print("reld11")
            encoded_prev_sentences = self.encode_sentences(prev_sentences)
            dist = np.array([self.get_sentence_distance(encoded_sentence, s) for s in encoded_prev_sentences])
            # print("reld112")
            indeces = np.where(dist > self.threshold)
            relevant_history = [self.history[i] for i in indeces[0]]
            # print("reld1122")
        demos = self.demos + ([self.get_demo(h) for h in relevant_history])
        # print("reld11111")
        return demos

    def concat_sentences(self, sentences):
        r = ""
        for s in sentences:
            if s.find("Options:") != -1:
                return None
            r = r + " " + s
        return r

    def combine_sentences(self, sentences, orig):
        i = 0
        n = len(sentences)
        s = []
        o = []
        while i < n:
            u = np.random.uniform()
            if u < self.combine_prob:
                r = self.concat_sentences(sentences[i: i + 2])
                ro = self.concat_sentences(orig[i: i + 2])
                if r is not None:
                    s.append(r)
                    o.append(ro)
                    i += 2
            else:
              s.append(sentences[i])
              o.append(orig[i])
              i += 1
        return s, o

    def select_sentence(self, prompt) -> int:
        sentences_orig = self.get_sentences(prompt)
        sentences = self.get_sentences(prompt, True)
        if len(self.history) < 1:
            flag = False
            while(not flag):
                self.index = np.random.randint(0, len(sentences))
                self.before = sentences[self.index]
                flag = self.before.find("Options: ") == -1
            return self.index

        # flag = False
        # while(not flag):
        prev_sentences = [h["before"] for h in self.history]
        r = np.array([h["score"] for h in self.history])
        h = self.encode_sentences(prev_sentences)
        n = h.shape[1]
        print(h.shape)
        a = np.dot(h.T, h) + self.lamda * np.identity(n)
        # print("a1")
        ainv = np.linalg.inv(a)
        w = np.dot(np.dot(ainv, h.T), r)
        # print("a11")
        scores = np.array([self.get_bandit_score(s, w, ainv) for s in h])
        u = np.random.uniform()
        if u < self.prob_select:
            index = np.where(scores == np.max(scores))[0][0]
            # print("a111")
        else:
            index = np.random.random_integers(0, len(scores))
        self.before = sentences[index]
        self.index = index
            # flag = self.before.find("Options:") == -1
        # print("a1111")
        return index

    def mutate_sentence(self) -> str:
        print("Selected:", self.before)
        prompt = "paraphraze: " + self.before
        print(prompt)
        input_ids = self.tokenizer(prompt, return_tensors = "pt").input_ids
        out = self.model.generate(input_ids, max_new_tokens = 1024)
        self.after = self.tokenizer.decode(out[0], skip_special_tokens = True)
        print("Mutated:", self.after)
        return self.after

    def update_history(self, scorediff):
        self.history.append({
            "before": self.before,
            "after": self.after,
            "score": scorediff
        })


    def update_mutator(self, scorediff):
        self.update_history(scorediff)

    def get_mutated_prompt(self, prompt) -> str:
        select_index = self.select_sentence(prompt)
        mutated_sentence = self.mutate_sentence()
        sentences = self.extract_sentences(prompt)
        if sentences[select_index].find("Question: ") != -1:
            mutated_sentence = "Question: " + mutated_sentence
            mutated_sentence = mutated_sentence.strip(".") + "?"
        elif sentences[select_index].find("Answer: ") != -1:
            mutated_sentence = "Answer: " + mutated_sentence

        sentences[select_index] = mutated_sentence
        return " ".join(sentences)

    def save_history(self, filename):
        pd.DataFrame(self.history).to_csv(filename)


In [ ]:
mutator = Mutator("chatgpt", combine_prob = 0.5)
instruction = "Answer questions about causal attribution."
demos = [
    {
        "question": "How would a typical person answer each of the following questions about causation?\nFrank T., had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body. Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild. Nonetheless, the bullet bounced off a large boulder several feet away and hit the neighbor's body, causing significant injury. Did Frank T. intentionally shoot his neighbor in the body?\nOptions:\n- Yes\n- No\n",
        "answer": "Let\'s think step by step.\nHere in this question, we are told that \"Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild.\" A typical person would assume that this passage suggests that Frank T. had no intention of shooting and injuring someone and that the bullet accidentally hit the neighbor\'s body; therefore, we conclude that Frank T. did not intentionally hit his neighbor. So the answer is No.\n\n"
    },
    {
        "question": "How would a typical person answer each of the following questions about causation?\nFrank T., had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body. Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild. Nonetheless, the bullet bounced off a large boulder several feet away and hit the neighbor's body, causing significant injury. Did Frank T. intentionally shoot his neighbor in the body?\nOptions:\n- Yes\n- No\n",
        "answer": "Let\'s think step by step.\nHere in this question, we are told that \"Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild.\" A typical person would assume that this passage suggests that Frank T. had no intention of shooting and injuring someone and that the bullet accidentally hit the neighbor\'s body; therefore, we conclude that Frank T. did not intentionally hit his neighbor. So the answer is No.\n\n"
    }
]
question = "How would a typical person answer each of the following questions about causation?\nA machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a short circuit. Did the black wire cause the short circuit?\nOptions:\n- Yes\n- No"



config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
context = open("./initial_prompts/causal_judgement").read()
t = Template("./templates/prompt/context", "./templates/demo/context").get_llm_input(question = question, context = context)
s = mutator.extract_sentences(t, False)
pprint(len(s))

49


In [ ]:
%time
print(mutator.get_mutated_prompt(t))
# mutator.update_history(-0.1)
# mutator.update_history(0.1)
# print(mutator.get_mutated_prompt(t))
# mutator.save_history("history.csv")

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs
Selected: Later in the afternoon they play a game of darts.
paraphraze: Later in the afternoon they play a game of darts.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Mutated: They engage in a game of darts in the late afternoon.
Answer questions about causal attribution. Question: How would a typical person answer each of the following questions about causation? Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body. Frank T had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild. Nonetheless, the bullet bounced off a large boulder several feet away and hit the neighbor's body, causing significant injury. Did Frank T intentionally shoot his neighbor in the body? Options: - Yes - No Answer: Let's think step by step. Here in this question, we are told that "Frank T had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild". A typical person would assume that this passage suggests that Frank T had no intention of shooting and injuring someone and that the bullet accidentally hit the neighbor's body; theref

In [ ]:
class Dataset:
    def __init__(self, format = "json", filepath = None, dict_data = None, template: Template = None):
        try:
            if format == "json":
                f = open(filepath)
                data_dict = json.load(f)
                self.df = pd.DataFrame(data_dict["examples"])
            elif format == "csv":
                self.df = pd.read_csv(filepath)
            elif format == "dict":
                if not isinstance(dict_data, list) and isinstance(dict_data["input"], str):
                    d = {k: [v] for k, v in dict_data.items()}
                else:
                    d = dict_data
                self.df = pd.DataFrame(d)
        except:
            raise FileNotFoundError("File not found or format not supported. Only json and csv is supported.", filepath)
        print(self.df.shape)
        if (template is None) or (not isinstance(template, Template)):
            raise ValueError("Invalid template. It should be an instance of class Template")
        self.template = template

    def get_data_as_df(self):
        return self.df

    def apply_template_ques(self, prompt, question):
        return self.template.get_llm_input(question = question, context = prompt)

    def apply_template_df(self, prompt, df = None) -> pd.DataFrame:
        if df is None:
            df = self.df.copy()
        df["input"] = df["input"].map(lambda x: self.template.get_llm_input(question = x, context = prompt))
        return df

    def get_random_sample(self, sample_size = 2):
        if sample_size > self.df.shape[0]:
            print("warning sample size largee than total samples. Returning the entire dataset")
            return self.df
        return self.df.sample(sample_size, replace = False).reset_index(drop = True)

    def get_tokenized_df(self, prompt, tokenizer, max_length):
        df = self.apply_template_df(prompt)
        df["input"] = df["input"].map(lambda x: tokenizer(x, padding = "longest"))
        return df


In [ ]:
file = "./datasets/causal_judgement.json"
prompt = "Some PRompt. \n aofb \n noafo.. $ aobfoa\n"
template = Template("./templates/prompt/context", "./templates/demo/context")
dataset = Dataset(format = "json", filepath = file, template = template)
print(dataset.get_data_as_df().head())
print(dataset.apply_template_df("Some PRompt. \n aofb \n noafo.. $ aobfoa\n")["input"][0])
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
df = dataset.get_tokenized_df(prompt, tokenizer, 1024)
df["input"][0]
print(df.head())


(187, 2)
                                               input target
0  How would a typical person answer each of the ...     No
1  How would a typical person answer each of the ...     No
2  How would a typical person answer each of the ...    Yes
3  How would a typical person answer each of the ...     No
4  How would a typical person answer each of the ...    Yes
Some PRompt. 
 aofb 
 noafo.. $ aobfoa


Question: How would a typical person answer each of the following questions about causation?
A machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a short circuit. Did 

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (640 > 512). Running this sequence through the model will result in indexing errors


                         input target
0  [input_ids, attention_mask]     No
1  [input_ids, attention_mask]     No
2  [input_ids, attention_mask]    Yes
3  [input_ids, attention_mask]     No
4  [input_ids, attention_mask]    Yes


In [ ]:
data_dict = [{
    "input": "How would a typical person answer each of the following questions about causation?\nA machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a short circuit. Did the black wire cause the short circuit?\nOptions:\n- Yes\n- No",
    "target": "No"
    }]
prompt = "Some PRompt. \n aofb \n noafo.. $ aobfoa\n"
template = Template("./templates/prompt/context", "./templates/demo/context")
dataset = Dataset(format = "dict", dict_data = data_dict, template = template)
print(dataset.get_data_as_df().head())
print(dataset.get_data_as_df()["input"][0])

(1, 2)
                                               input target
0  How would a typical person answer each of the ...     No
How would a typical person answer each of the following questions about causation?
A machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a short circuit. Did the black wire cause the short circuit?
Options:
- Yes
- No


In [ ]:
# from openai import OpenAI
dataset.get_random_sample(2)

warning sample size largee than total samples. Returning the entire dataset


,input,target
0,How would a typical person answer each of the ...,No


In [ ]:
from pprint import pprint
class GA:
    supported_models = ["t5-small", "t5-base", "t5-large", "gpt2"]
    def __init__(self, model, dataset: Dataset, mutator, fitness_func = None, num_iter = 100, k = 5, select_method = "random", max_length = 512, batch_size = 2, sample_size = None):
        if model not in self.supported_models:
            raise ValueError("Model mot suported. Supported models are", self.supported_models)
        elif model == "t5-small":
            checkpoint = "google/flan-t5-small"
        elif model == "t5-base":
            checkpoint = "google/flan-t5-base"
        elif model == "t5-large":
            checkpoint = "google/flan-t5-large"
        else:
            checkpoint = "distilgpt2"

        self.max_length = max_length
        self.model = get_model(checkpoint)
        self.tokenizer = get_tokenizer(checkpoint)

        self.fitness_function = fitness_func


        # self.template = Template(template, demo_template)
        self.k = k
        self.num_iter = num_iter
        self.batch_size = batch_size
        self.select_method = "random"
        self.has_run = False
        self.mutator = mutator
        self.dataset = dataset
        if sample_size is None:
            self.df = dataset.get_data_as_df()
        else:
            self.df = dataset.get_random_sample(sample_size)
        self.init_prompt = open("./initial_prompts/causal_judgement").read()
        self.init_prompt = {
                "prompt": self.init_prompt,
                "score": 0
            }
        # self.pop = [
        #     self.prompt
        # ]

    def select_random(self, pop):
        return np.random.randint(0, len(pop))

    def select_weighted(self, pop):
        if len(pop) == 1:
            return 0
        else:
            return np.random.randint(0, len(pop), p = self.scores)

    def select_prompt(self, pop):
        if self.select_method == "random":
            return self.select_random(pop)
        elif self.select_method == "weighted":
            return self.select_weighted(pop)


    def get_best_candidate(self):
        return sorted(self.pop, key = lambda x: x["score"], reverse = True)[0]

    def get_sentence_distance(self, a, b):
        ea = self.mutator.encode_sentences(a)
        eb = self.mutator.encode_sentences(b)
        return self.mutator.get_sentence_distance(ea, eb)

    def select_crossover_point(self, a, b):
        max_sim = 10
        i = 0
        n = min(len(a), len(b))
        pos = np.random.randint(0, n - 1)
        while self.get_sentence_distance(a[pos], b[pos]) < 0.5 and i < max_sim:
            pos = np.random.randint(0, n - 1)
            i += 1
        return pos

    def convert_to_prompt(self, sentences):
        return " ".join(sentences)

    def get_children(self, a, b):
        sa = self.mutator.get_sentences(a)
        sb = self.mutator.get_sentences(b)
        pos = self.select_crossover_point(sa, sb)
        return self.convert_to_prompt(sa[:pos] + sb[pos:]), self.convert_to_prompt(sb[:pos] + sa[pos:])

    def get_prompt_dict(self, prompt):
        return {
            "prompt": prompt,
            "score": 0
        }

    def crossover(self):
        best_prompt = self.get_best_candidate()

        new_pop = [best_prompt]
        for i in range(self.k // 2):
            a, b = np.random.choice(self.pop, size = 2, replace = False)
            ca, cb = self.get_children(a["prompt"], b["prompt"])
            ca = self.get_prompt_dict(ca)
            cb = self.get_prompt_dict(cb)
            new_pop.extend([ca, cb])
        return new_pop

    def mutate_prompt(self, prompt):
        mutated_prompt = self.mutator.get_mutated_prompt(prompt["prompt"])
        return self.get_prompt_dict(mutated_prompt)

    def init_population(self):
        self.pop = [self.mutate_prompt(self.init_prompt) for _ in range(self.k)]

    def generate_answers(self, prompt: str, df_orig):
        if df_orig is None:
            df = self.dataset.apply_template_df(prompt, self.df)
        else:
            df = self.dataset.apply_template_df(prompt, df_orig)
        n = df.shape[0]
        inputs = []
        for i in range(n):
            inputs.append(self.tokenizer(df["input"][i], max_length = self.max_length))

        generated_text = []
        for ip in inputs:
            out = self.model.generate(torch.tensor([ip["input_ids"]]), max_new_tokens = 1024)
            r = self.tokenizer.decode(out[0], skip_special_tokens = True)
            generated_text.append(r)
        answers = [extract_answer(s) for s in generated_text]
        answers = np.array(list(map(encode_answer, answers)))
        target = np.array(list(map(encode_answer, df["target"])))
        for i, a in enumerate(answers):
          if a is None:
            answers[i] = !target[i]
        return pd.DataFrame({
            "answers": answers,
            "target": target
        })

    def evaluate_fitness(self, prompt: str, df = None):
        if callable(self.fitness_function):
            try:
                ans = self.fitness_function(prompt, df)
                ans = float(ans)
                return ans
            except AssertionError:
                raise ValueError("The output of the fitness function must be convertible to a numeric value example int or float")
            except AttributeError:
                raise AttributeError("Did you accidently included the parentheses")

        ans_df = self.generate_answers(prompt, df)
        return (ans_df["answers"] == ans_df["target"]).mean()

    def evaluate_population(self, pop):
        for prompt in pop:
            prompt["score"] = self.evaluate_fitness(prompt["prompt"])
        return

    def update_population(self, new_pop):
        # print(type(new_pop))
        self.pop.extend(new_pop)
        self.pop = sorted(self.pop, key = lambda x: x["score"], reverse = True)
        self.pop = self.pop[:self.k]
        # print(self.pop[0])
        # print(type(self.pop[0]))

    def tune_prompt(self):
        self.init_score = self.evaluate_fitness(self.init_prompt["prompt"])
        self.init_population()
        self.evaluate_population(self.pop)
        for i in range(self.num_iter):
            print("iteration:", i)
            new_pop = self.crossover()
            print("crossove done.")
            index = self.select_prompt(new_pop)
            print("selecting and mutating a random prompt")
            init_score = self.evaluate_fitness(new_pop[index]["prompt"]) # expensive
            new_pop[index] = self.mutate_prompt(new_pop[index])
            self.evaluate_population(new_pop) # expensive
            score = new_pop[index]["score"]
            self.mutator.update_mutator(score - init_score)
            self.update_population(new_pop)
            print("Scores:", [p["score"] for p in self.pop])

    def best_prompt(self):
        self.tune_prompt()
        try:
            self.mutator.save_history("history.csv")
        except:
            print("Could not save history")
        max_score = self.pop[0]["score"]
        best_prompt = self.pop[0]
        for p in self.pop:
            if max_score < p["score"]:
                max_score = p["score"]
                best_prompt = p
        return best_prompt

    def save_best_prompt(self, filename = "best_prompt.txt"):
        best_prompt = self.best_prompt()
        print(best_prompt["prompt"])
        initial_score = self.evaluate_fitness(self.init_prompt["prompt"], self.dataset.df)
        print("Score: ", initial_score)
        best_score = self.evaluate_fitness(best_prompt["prompt"], self.dataset.df)
        print("Score: ", best_score)
        self.has_run = True
        try:
            f = open(filename, mode = 'w')
            f.write(best_prompt["prompt"])
            f.close()
        except:
            pass
        return best_prompt["prompt"], best_score

    def answer_question(self, question):
        if not self.has_run:
            print("The algorithm has not been run. Using the initial prompt")
            prompt = self.init_prompt["prompt"]
        else:
          prompt = open("best_prompt.txt").read()
        prompt = self.dataset.apply_template_ques(question = question, context = prompt)
        return self



In [ ]:
template = Template("./templates/prompt/context", "./templates/demo/context")
dataset = Dataset(format = "json", filepath = "./datasets/causal_judgement.json", template = template)
mutator = Mutator("chatgpt")
ga = GA("t5-small", dataset, mutator = mutator, max_length = 1024 + 512, num_iter = 50, sample_size = 10)

(187, 2)


In [ ]:
ga.evaluate_fitness(ga.init_prompt["prompt"], dataset.df)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


0.43315508021390375

In [ ]:
# ga.generate_answers()
ga.save_best_prompt("best_prompt.txt")

Selected: Here in this question, we are told that "He aims the dart at the low point region".
paraphraze: Here in this question, we are told that "He aims the dart at the low point region".


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Mutated: The question entails the statement that "He directs the ball at the low area."
Selected: Both Billy and Suzy arrive at 9 am.
paraphraze: Both Billy and Suzy arrive at 9 am.
Mutated: Billy and Suzy both arrive at 9 am.
Selected: Nonetheless, the bullet bounced off a large boulder several feet away and hit the neighbor's body, causing significant injury.
paraphraze: Nonetheless, the bullet bounced off a large boulder several feet away and hit the neighbor's body, causing significant injury.
Mutated: Despite this, the bullet ricocheted off a big boulder several feet away and struck the neighbor's body, causing significant harm.
Selected: A typical person would assume that the person probably meant to say the detector was set up to be triggered if "both persons" appeared in the room at the same time, not at least one person, since otherwise the phrase "at the same time" would not make much sense in that sentence.
paraphraze: A typical person would assume that the person probably m

<ipython-input-10-cc52f143ce5e>:201: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  index = np.random.random_integers(0, len(scores))


Mutated: Answer queries concerning causal attribution.
Scores: [0.5, 0.5, 0.5, 0.5, 0.5]
iteration: 2
crossove done.
selecting and mutating a random prompt
(2, 384)
Selected: How would a typical person answer each of the following questions about causation?
paraphraze: How would a typical person answer each of the following questions about causation?
Mutated: What would be the typical response to the following questions regarding causalities in an average individual?
Scores: [0.5, 0.5, 0.5, 0.5, 0.5]
iteration: 3
crossove done.
selecting and mutating a random prompt
(3, 384)
Selected: Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body.
paraphraze: Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body.
Mutated: Frank T and his neighbor were in a constant argument over a land parcel, and he ultimately chose to shoot them in the head.
Scores:

<ipython-input-10-cc52f143ce5e>:201: DeprecationWarning: This function is deprecated. Please call randint(0, 4 + 1) instead
  index = np.random.random_integers(0, len(scores))


Mutated: What would be the typical response to the following questions regarding causalities in an average individual?
Scores: [0.5, 0.5, 0.5, 0.5, 0.5]
iteration: 5
crossove done.
selecting and mutating a random prompt
(5, 384)
Selected: Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body.
paraphraze: Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body.
Mutated: Frank T and his neighbor were in a constant argument over a land parcel, and he ultimately chose to shoot them in the head.
Scores: [0.5, 0.5, 0.5, 0.5, 0.5]
iteration: 6
crossove done.
selecting and mutating a random prompt
(6, 384)
Selected: Frank T had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild.
paraphraze: Frank T had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild.


<ipython-input-10-cc52f143ce5e>:201: DeprecationWarning: This function is deprecated. Please call randint(0, 6 + 1) instead
  index = np.random.random_integers(0, len(scores))


Mutated: Frank T's hand slipped on the gun's barrel, resulting in a shot that went off in the air.
Scores: [0.5, 0.5, 0.5, 0.5, 0.5]
iteration: 7
crossove done.
selecting and mutating a random prompt
(7, 384)
Selected: Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body.
paraphraze: Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body.
Mutated: Frank T and his neighbor were in a constant argument over a land parcel, and he ultimately chose to shoot them in the head.
Scores: [0.5, 0.5, 0.5, 0.5, 0.5]
iteration: 8
crossove done.
selecting and mutating a random prompt
(8, 384)
Selected: Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body.
paraphraze: Frank T, had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in th

In [ ]:
mutator.save_history("history.csv")

In [ ]:
"""
OpenDI HyperCycle Hackathon 2023
Challenge 3: Genetic Algorithm for Automated Prompt Engineering
"""
import os
import json

from pyhypercycle_aim import SimpleServer, JSONResponseCORS, aim_uri

# from ga import GA
# from mutator import Mutator
# from dataset import Dataset
# from template import Template

PORT = os.environ.get("PORT", 4002)


class GeneticWrapper(SimpleServer):
    manifest = {"name": "CBGeneticAlgo",
                "short_name": "cb-gen",
                "version": "0.1",
                "license": "MIT",
                "author": "TeamAC"
                }

    def __init__(self):
        pass

    @aim_uri(uri="/prompt", methods=["POST"],
             endpoint_manifest={
                 "input_query": "",
                 "input_headers": "",
                 "output": {},
                 "documentation": "Returns the prompt and the score based on the desired output",
                 "example_calls": [{
                     "body": {"data": [{"input": "How would a typical person answer each of the following questions about causation?\nA machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a short circuit. Did the black wire cause the short circuit?\nOptions:\n- Yes\n- No", "target": "No"}]},
                     "method": "POST",
                     "query": "",
                     "headers": "",
                     "output": {"prompt": "string", "score": 0.004}
                 }]
             })
    async def prompt(self, request):
        # define the total iterations
        n_iter = 100
        # define the population size
        n_pop = 100
        # define sample size to used during training
        sample_size = 4

        prompt_template = "[context]\n\nQuestion: [question]\nAnswer: Let's think step by step."
        demo_template = "Question: [question]\nAnswer: [answer]"
        template = Template(prompt_template, demo_template)
        request_json = await request.json()
        dt = request_json['data']
        dataset = Dataset(format = "dict", dict_data = json.loads(dt), template = template)
        mutator = Mutator()
        ga = GA("t5-small", dataset = dataset, mutator = mutator, num_iter = n_iter, k = n_pop)

        # perform the genetic algorithm search
        best_prompt, score = ga.save_best_prompt()
        return JSONResponseCORS({"prompt": best_prompt, "score": score})


def main():
    # example usage:
    app = GeneticWrapper()
    app.run(uvicorn_kwargs={"port": PORT, "host": "0.0.0.0"})


if __name__ == '__main__':
    main()